<a href="https://colab.research.google.com/github/Nyehsee/PythonProjects/blob/main/api2Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Default title text

import requests
import pandas as pd
from datetime import datetime
import plotly.express as px
from numpy import dot

###CYBER CREW WALLET KEY 0x9b944a5f2f90ba9f8a8c375be8b076b8361bff15
walletKey = "0x9b944a5f2f90ba9f8a8c375be8b076b8361bff15" #@param {type:"string"}

#get #s of collection
collectionNumUrl = 'https://api.nft.gamestop.com/nft-svc-marketplace/getCollectionsPaginated?limit=0&creatorEthAddress='
response = requests.get(collectionNumUrl+ walletKey)
jsonResponse = response.json()
totalCollectionNum = jsonResponse['totalNum']

#get collection details
collectionDetailsUrl = f'https://api.nft.gamestop.com/nft-svc-marketplace/getCollectionsPaginated?limit={totalCollectionNum}&offset=0&creatorEthAddress='
response = requests.get(collectionDetailsUrl+ walletKey)
jsonResponse = response.json()

collectionIdDict = []
for i in jsonResponse['data']:
    collectionId = i['collectionId']
    collectionIdDict.append(collectionId)

collectionDataDict = []
for id in collectionIdDict:
    r = requests.get(f'https://api.nft.gamestop.com/nft-svc-marketplace/getNftsPaginated?limit=48&offset=0&collectionId={id}&sortBy=created&sortOrder=desc')
    jR = r.json()
    collectionDataDict.append(jR)

nftDict = []
for i in collectionDataDict:
    for n in i['data']:
        nftDict.append([n['name'],n['loopringNftInfo']['nftData']])


def get_data(token_id, name, data_list=[]):
    URL = "https://api.nft.gamestop.com/nft-svc-marketplace/history?nftData="
    response = requests.get(URL + token_id)
    # response.raise_for_status()
    # access JSOn content
    jsonResponse = response.json()
    for block in jsonResponse:
        for key, value in block.items():
            if key == "createdAt":
                ts = int(value) / 1000.0  # Because its a Unix epoch with milliseconds
                # created = datetime.fromtimestamp(ts)
                created = datetime.utcfromtimestamp(ts)

            if key == "transaction":
                for k, v in value.items():
                    if k == "orderB":
                        qty = int(v.get("amountS"))
                        paid = int(v.get("amountB")) / 1e18
                        paid = paid / qty

                        data_list.append([created, paid, qty, name])
    return data_list


lst = []
for nft in nftDict:
    lst = get_data(nft[1][0], nft[0]) #nftData, name

# for i in nft_dict:
#     print(i[0], i[1][0])

dot_marker = True

df = pd.DataFrame(lst, columns=['Time GMT', 'Price ETH', 'Quanty', 'Name'])
fig = px.line(df, x="Time GMT", y="Price ETH", color='Name', markers=dot_marker, template="plotly_dark")
fig.show()
